In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import os

import json
from shapely.geometry import MultiPoint, Polygon, Point, LineString
from shapely.ops import unary_union
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib notebook
import osmnx as ox
import pyproj
from scipy import spatial



# sample geographical points for Copenhagen
This work is inspired by this paper --> https://arxiv.org/abs/2105.01764.
We extensively make use of their code from --> https://github.com/stanford-policylab/surveilling-surveillance/blob/master/streetview/sample.py

In [3]:
# load shapes for danish kommuner (regions)
shapes = json.load(open('data/denmark-kommuner.geojson'))

In [4]:
# find geocoordinates for the capital copenhagen (CPH)
cph = []
for s in shapes['features']:
    if s['properties']['KOMNAVN'] == 'København':
        cph.append(Polygon(s['geometry']['coordinates'][0]))
# take the unary union to also include Frederiksberg (which is a city inside the capital)
cph = unary_union(cph)

In [6]:
# download street network from OSM for all road segments in the polygon
G = ox.graph_from_polygon(cph,network_type='drive')

# extract position for nodes (intersections)
pos = dict(G.nodes(data = True))

__plot map - not necessary to run for below code to work__

In [7]:
# draw map
plt.figure(figsize=(10,10))
plt.style.use('dark_background')
for segment in G.edges(data=True):
    if 'geometry' in segment[2]:
        x,y = segment[2]['geometry'].xy
        plt.plot(x,y,color='white',lw=0.5)
    else:
        i,j = segment[:2]
        xi = pos[i]['x']; yi = pos[i]['y']
        xj = pos[j]['x']; yj = pos[j]['y']
        plt.plot([xi,xj],[yi,yj],color='white',lw=0.5)
plt.axis('off')
plt.savefig('plots/copenhagen-roads.png',dpi=300)
plt.show()

<IPython.core.display.Javascript object>

# sample points from street network

### Sample uniformly from polygon - probably not the best approach.
As we will be forced to map points to their closest road, and roads next to lakes, parks and other places with no roads will be overrepresented.

In [8]:
# as official polygon contains islands into the sea which have no roads, we find bbox of polygon from pos of nodes
x_min = 180; x_max = -180
y_min = 90; y_max = -90
for i in pos.values():
    x_min = min(x_min,i['x'])
    x_max = max(x_max,i['x'])
    y_min = min(y_min,i['y'])
    y_max = max(y_max,i['y'])
bbox = (x_min, x_max, y_min, y_max)

In [9]:
def generate_random_points(polygon, no_points, bbox):
    points = []
    while len(points) < no_points:
        # generate random point in bbox (x_min, x_max, y_min, y_max)
        p = Point(np.random.uniform(x_min, x_max), np.random.uniform(y_min, y_max))
        if polygon.contains(p):
            points.append((p.x,p.y))
    return points

In [10]:
points = generate_random_points(cph,10000,bbox)

In [11]:
# Plot the polygon
plt.figure()
for p in cph.geoms:
    xp,yp = p.exterior.xy
    plt.plot(xp,yp,color='white')

# Plot the list of points
xs,ys = zip(*points)
plt.scatter(xs, ys, s=4, color="red")
plt.show()

<IPython.core.display.Javascript object>

### Second attempt: First sample roads, then sample points from roads

In [12]:
def get_pos_for_node(node):
    return Point(pos[node]['x'], pos[node]['y'])

def sample_points(n):
    # first transform edges to dataframe
    df = []
    for i,j,info_ in list(G.edges(data=True)):

        # if geometry exists for road (i.e. its not just a straight line)
        if 'geometry' in info_:
            #df.append((i,j,get_pos_for_node(i),get_pos_for_node(j),info_['length'],info_.get('geometry','None')))
            df.append((i,j,info_['length'],info_['geometry']))
        # if its a straight line we need to add geometry
        else:
            df.append((i,j,info_['length'],LineString([get_pos_for_node(i),get_pos_for_node(j)])))
    df = pd.DataFrame(df, columns = ['source','target','length','geometry'])

    # number of edges/roads
    m = df.shape[0]
    # total sum of road length
    total_length = sum(df['length'])
    # length normalized
    df['normalized_length'] = df['length'].div(total_length)

    # select random roads, weighted according to their length
    indices = np.random.choice(range(m),size=n, p=df['normalized_length'])
    
    # define geodesic distance function - to be used for finding road bearings
    geodesic = pyproj.Geod(ellps='WGS84')

    # use tqdm to print progress
    pbar = tqdm(total=n)
    # store data in rows
    rows = []
    
    # sample n points from road segments
    i = 0
    while i < n:
        # unpack random row/road segment
        index = indices[i]
        row = df.iloc[index]
        line = row['geometry']
        # pick a random place on the segment
        delta = np.random.rand() * line.length
        # find the coordinates this corresponds to
        point = line.interpolate(delta)
        lat = point.y
        lon = point.x

        # update counters
        i += 1
        pbar.update(1)
        
        # do start and end of point to calculate heading
        start = line.interpolate(delta*0.9)
        end = line.interpolate(min(line.length, delta*1.1))
        # find heading using the geodesic function
        fwd_azimuth,back_azimuth,_ = geodesic.inv(start.x, start.y, end.x, end.y)  #geodesic.inv(start.y, start.x, end.y, end.x)
        
        # store in rows
        rows.append({"lat": lat,
                     "lon": lon,
                     "id": i,
                     "heading": fwd_azimuth,
                     "delta": delta})
    
    # stop the progress bar
    pbar.close()

    # return samples points in dataframe
    return pd.DataFrame(rows)

In [13]:
points = sample_points(150000)

100%|██████████| 150000/150000 [00:21<00:00, 6965.33it/s]


__plot sampled points__

In [14]:
# draw map
plt.figure(figsize=(10,10))
plt.style.use('dark_background')
# draw road segments
for segment in G.edges(data=True):
    if 'geometry' in segment[2]:
        x,y = segment[2]['geometry'].xy
        plt.plot(x,y,color='white',lw=0.5)
    else:
        i,j = segment[:2]
        xi = pos[i]['x']; yi = pos[i]['y']
        xj = pos[j]['x']; yj = pos[j]['y']
        plt.plot([xi,xj],[yi,yj],color='white',lw=0.5)
# plot samples points
plt.scatter(points['lon'],points['lat'],s=2,color='red',zorder=4)
        
plt.axis('off')
plt.savefig('plots/copenhagen-roads-with-150k-sampled-points.png',dpi=300)
plt.show()

<IPython.core.display.Javascript object>

# clean up points
Remove points which are too close to each other - i.e. less than 7 meters (somewhat arbitrarily set)

In [15]:
# create transformation to transform from GPS (lat,lon) coordinates to UTM coordinates (i.e. given in meters)
transformation = pyproj.Proj(proj='utm', zone=33, ellps='WGS84')

# transform datapoints
x,y = transformation(points['lon'],points['lat'])

# build spatial tree
tree = spatial.KDTree(np.array((x,y)).T)

In [16]:
# query tree
candidates_for_deletion = set()
# iterate over points
for i in range(points.shape[0]):
    # if point has not already been flaged for deletion
    if i not in candidates_for_deletion:
        # fidn nearby points
        nearby = tree.query_ball_point((x[i],y[i]),r=7)
        # if there is more than the point itself 
        if len(nearby) > 1:
            # iterate over nearby points
            for j in nearby:
                # if not the point itself
                if j!=i:
                    # flag for removal
                    candidates_for_deletion.add(j)

In [17]:
# remove close points
points = points[~points['id'].isin(candidates_for_deletion)]

# save sampled points to csv
points.to_csv('data/sampled-points-150k.csv',index=False)

In [18]:
   
def select_panoid(meta, 
                    n=5000, 
                    distance=10, 
                    selection="closest",
                    seed=123):
    YEARS = ["2020<year<2023", "2023<=year"]
    
    # Set random seed
    np.random.seed(seed)
    random.seed(seed)

    # Filter by distance
    meta = meta.query(f"distance < {distance}")
    
    # Filter by occurance for both pre and post
    meta_pre = meta.query(YEARS[0]).drop_duplicates(["lat_anchor", "lon_anchor"])
    meta_post = meta.query(YEARS[1]).drop_duplicates(["lat_anchor", "lon_anchor"])    
    meta_both = meta_pre.merge(meta_post, on=["lat_anchor", "lon_anchor"], how="inner")
    
    # Sample anchor points 
    meta_sample = meta_both.drop_duplicates(['lat_anchor', 'lon_anchor']).sample(n, replace=False)
    lat_anchor_chosen = meta_sample.lat_anchor.unique()
    lon_anchor_chosen = meta_sample.lon_anchor.unique()

    # Sample for pre and post
    meta_sub = meta[meta.lat_anchor.isin(lat_anchor_chosen)]
    meta_sub = meta_sub[meta_sub.lon_anchor.isin(lon_anchor_chosen)]

    # Select panoid
    groups = []
    for years in YEARS:
        group = meta_sub.query(years)
        if selection == "closest":
            group = group.sort_values(['lat_anchor','lon_anchor', 'distance']) 
        else:
            group = group.sort_values(['lat_anchor','lon_anchor', 'year'], ascending=False) 
        group = group.groupby(['lat_anchor','lon_anchor']).first().reset_index()        
        group['year'] = group.year.apply(int)
        groups.append(group)
    
    # Random select the orthogonal heading
    merged = groups[0].merge(groups[1], 
                             on=['lat_anchor', 'lon_anchor', 'u', 'v', 'key', 'heading', 'offset'], 
                             suffixes=("_pre", "_post"))
    
    merged['heading_pre'] = merged['heading_post'] = (merged.heading + 360 + 90 - 180 * (np.random.rand(n) > 0.5)) % 360
    merged['heading_pre'] = merged['heading_pre'].apply(int)
    merged['heading_post'] = merged['heading_post'].apply(int)
    return merged

def select_panoid_recent(meta,
                    year,
                    n=5000, 
                    distance=10,
                    seed=123):
    
    # Set random seed
    np.random.seed(seed)
    random.seed(seed)

    # Filter by distance
    meta = meta.query(f"distance < {distance}")
    meta = meta.query(f"year >= {year}")

    # Sample anchor points 
    meta_sample = meta.drop_duplicates(['id']).sample(n, replace=False)
    lat_anchor_chosen = meta_sample.lat_anchor.unique()
    lon_anchor_chosen = meta_sample.lon_anchor.unique()

    # Sample for pre and post
    meta_sub = meta[meta.lat_anchor.isin(lat_anchor_chosen)]
    meta_sub = meta_sub[meta_sub.lon_anchor.isin(lon_anchor_chosen)]

    # Select panoid

    meta = meta_sub.sort_values(['lat_anchor','lon_anchor', 'distance']) \
                         .groupby(['lat_anchor','lon_anchor']) \
                         .first().reset_index()     

    # Random select the orthogonal heading
    meta['road_heading'] = meta.heading
    meta['heading'] = (meta.heading + 360 + 90 - 180 * (np.random.rand(n) > 0.5)) % 360
    meta['heading'] = meta['heading'].apply(int)
    meta['year'] = meta['year'].apply(int)
    meta['month'] = meta['month'].apply(int)
    meta['save_path'] = meta.apply(get_path, 1)
    return meta

def get_path(row):
    panoid = row['panoid']
    heading = row['heading']
    return os.path.join("/scratch/haosheng/camera/", panoid[:2], panoid[2:4], panoid[4:6], panoid[6:], f"{heading}.png")
